In [1]:
import pandas as pd
import tkinter as tk
from tkinter import filedialog
pd.set_option("display.max_columns", None)
pd.set_option("mode.chained_assignment", None)

In [2]:
root = tk.Tk()
root.withdraw()
file_path = filedialog.askopenfilename()

In [3]:
class SizeDataTransformer:
    
    def __init__(self, path):
        self.path = path
        
    def read_data(self):
        if self.path.split(".")[-1] == "xlsx":
            df = pd.read_excel(self.path, None)
            catalog = {}
            for i, j in enumerate(list(df.keys())):
                catalog.setdefault(i + 1, j)
                print(i + 1, j)
            answer = int(input("Please select the sheet"))
            df = df[catalog[answer]]
        else:
            df = pd.read_csv(self.path)
        return df
    
    @staticmethod
    def tire_type(df, category, channel=None):
        df = df[(df["KA分類"] == category) & (df["大通路分類"] == channel) & (df["年月"].astype(str).str.contains("2022|2023"))]
        df["整理後的日期"] = pd.to_datetime(df["年月"], format="%Y%m")
        return df
        
        
        
    
    

In [4]:
df = SizeDataTransformer(file_path)

In [5]:
df = df.read_data()

1 SIZE DATA


Please select the sheet 1


In [6]:
result = SizeDataTransformer.tire_type(df, "TBR(G)", "REP")

In [7]:
result["條數"] = result["條數"].fillna(0)

In [8]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31368 entries, 90420 to 608023
Data columns (total 35 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   年度        31368 non-null  int64         
 1   季度        31368 non-null  object        
 2   月份        31368 non-null  object        
 3   年月        31368 non-null  int64         
 4   種類        31368 non-null  object        
 5   客戶類別      11552 non-null  object        
 6   客戶契約      8770 non-null   object        
 7   通路明細      31368 non-null  object        
 8   客戶代號      8770 non-null   object        
 9   客戶名稱      30048 non-null  object        
 10  營業所       31368 non-null  object        
 11  舊分類       31368 non-null  object        
 12  國別　       31368 non-null  object        
 13  商品代號      31368 non-null  object        
 14  規格        31368 non-null  object        
 15  條數        31368 non-null  float64       
 16  Gross     31368 non-null  float64       
 17  discoun

In [9]:
result.to_excel(r"D:\kc.hsu\OneDrive - Bridgestone\數據\2022ACT\2021-2022_Size Data_All_TBR_rawling_data.xlsx", index=False)